In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession
from pyspark.mllib.classification import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from time import perf_counter

In [2]:
NUM_NODES=4
# when set local[*], all available cores will be used, the only executor is the driver
conf = SparkConf().setMaster("spark://10.99.250.19:7077").setAppName("HIGGS") 
conf.set("spark.executor.memory", "20G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "8")
conf.set("spark.cores.max", f"{NUM_NODES*8}")
# conf.set("spark.executor.instances", "2")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
# conf.set("spark.default.parallelism", "16")
# create a Spark Session instead of a Spark Context
spark = SparkSession.builder \
    .config(conf = conf) \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/05 15:14:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_list = []
for i in range(NUM_NODES):
    df_list.append(spark.read.option("delimiter", ",").option("header", "false").csv('/work/li.baol/data/HIGGS.csv'))
if NUM_NODES == 1:
    df = df_list[0]
else:
    df = df_list[0]
    for k in range(1,NUM_NODES):
        df = df.union(df_list[k])

In [4]:
data = df.rdd.map(lambda row: LabeledPoint(float(row['_c0']), list(map(lambda x: float(x), row.asDict().values()))[1:]))

In [5]:
start = perf_counter()

model = DecisionTree.trainClassifier(data, numClasses=2, categoricalFeaturesInfo={},
                                    impurity='gini', maxDepth=8)
end = perf_counter()
end-start                   

99.06489806156605